Day 4 3 Transfer learning

In [1]:
import torch
import torchvision
from tqdm import tqdm
import gc
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Step 1) Dataset preparation
train_path = './tomato/train'
test_path = './tomato/val'

# create an empty list
transform = [torchvision.transforms.Resize((256,256)),
 torchvision.transforms.ToTensor(),
 torchvision.transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])]

transformation = torchvision.transforms.Compose(transform)

train_dataset = torchvision.datasets.ImageFolder(root=train_path,
                                                transform=transformation)
test_dataset = torchvision.datasets.ImageFolder(root=test_path,
                                                transform=transformation)
print(len(train_dataset))
print(len(test_dataset))
train_dataset[0][0].shape
train_dataset.classes

10000
1000


['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

In [3]:
print(device)

cuda:0


In [ ]:
batch_size = 4
num_epochs = 30
learning_rate = 0.0001
num_classes = 10


In [5]:
train_loader=torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
)
test_loader=torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
)






In [6]:
model=torchvision.models.efficientnet_b3(pretrained=False)
n_inputs=model.classifier[1].in_features
# print(model)
print(n_inputs)
model.classifier[1]=torch.nn.Linear(n_inputs,10)

creiterion=torch.nn.CrossEntropyLoss()
optim=torch.optim.Adam(model.parameters(),lr=0.0001)


C:\Users\User\anaconda3\envs\cpuenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User\anaconda3\envs\cpuenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


1536


In [7]:
def fit(epoch, model, trainloader, testloader):
    correct = 0
    total = 0
    running_loss = 0
    model.to(device)
    model.train()
    for x, y in tqdm(trainloader):
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        loss = creiterion(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()

    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total

    test_correct = 0
    test_total = 0
    test_running_loss = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(testloader):
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x.to(device))
            loss = creiterion(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()

    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total

    print('epoch: ', epoch,
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'test_loss： ', round(epoch_test_loss, 3),
          'test_accuracy:', round(epoch_test_acc, 3)
          )

    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc


In [8]:
# Initialize lists for storing metrics
train_loss = []
train_acc = []
test_loss = []
test_acc = []

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Timestamp', 'Epoch', 'Train_Loss', 'Train_Acc', 'Test_Loss', 'Test_Acc'])

for epoch in range(num_epochs):
    gc.collect()
    torch.cuda.empty_cache()
    epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch,
                                                                 model,
                                                                 train_loader,
                                                                 test_loader)
    
    # Get current timestamp
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Save the model with a timestamp
    model_filename = f"project_efficient_{timestamp.replace(':', '').replace(' ', '_')}.pt"
    torch.save(model.state_dict(), model_filename)
    
    # Append metrics to the lists
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)
    
    # Add the epoch results to the DataFrame
    new_row = {
        'Timestamp': timestamp,
        'Epoch': epoch + 1,
        'Train_Loss': epoch_loss,
        'Train_Acc': epoch_acc,
        'Test_Loss': epoch_test_loss,
        'Test_Acc': epoch_test_acc
    }

    new_row_df = pd.DataFrame([new_row])
    results_df = pd.concat([results_df, new_row_df], ignore_index=True)
    
    # Save the DataFrame to a CSV file with a timestamped filename
    csv_filename = f"training_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    results_df.to_csv(csv_filename, index=False)



100%|████████████████████████████████████████████████████████████████████████████| 250/250 [00:33<00:00,  7.50it/s]
C:\Users\User\AppData\Local\Temp\ipykernel_9956\3204854842.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row_df], ignore_index=True)


epoch:  0 loss：  0.487 accuracy: 0.293 test_loss：  0.303 test_accuracy: 0.585


100%|████████████████████████████████████████████████████████████████████████████| 250/250 [00:12<00:00, 20.78it/s]


epoch:  1 loss：  0.335 accuracy: 0.537 test_loss：  0.236 test_accuracy: 0.662


100%|████████████████████████████████████████████████████████████████████████████| 250/250 [00:11<00:00, 20.87it/s]


...

epoch:  27 loss：  0.022 accuracy: 0.974 test_loss：  0.037 test_accuracy: 0.955


100%|████████████████████████████████████████████████████████████████████████████| 250/250 [00:11<00:00, 20.96it/s]


epoch:  28 loss：  0.021 accuracy: 0.975 test_loss：  0.053 test_accuracy: 0.936


100%|████████████████████████████████████████████████████████████████████████████| 250/250 [00:11<00:00, 20.94it/s]

epoch:  29 loss：  0.02 accuracy: 0.977 test_loss：  0.056 test_accuracy: 0.937
